In [1]:
import deepmatcher as dm
import pandas as pd
import numpy as np
from scipy.spatial import distance
import math
from torch.autograd import Variable

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
data_dir = 'sample_data/itunes-amazon'
datasets = dm.data.process(data_dir, train='train.csv', validation='validation.csv',
                           test='price-test.csv')
train = datasets[0]
validation = datasets[1]
test = datasets[2]

In [4]:
train_table = train.get_raw_table()
train_table.head()

,id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
0,448,0,baby when the light ( david guetta & fred rist...,david guetta,pop life ( extended version ) [ bonus version ],"dance , music , rock , pop , house , electroni...",$ 1.29,‰ ãñ 2007 gum records,6:17,18-sep-07,revolver ( madonna vs. david guetta feat . lil...,david guetta,one love ( deluxe version ),dance & electronic,$ 1.29,( c ) 2014 swedish house mafia holdings ltd ( ...,3:18,"august 21 , 2009"
1,287,1,outversion,mark ronson,version,"pop , music , r & b / soul , soul , dance , ro...",$ 0.99,2007 mark ronson under exclusive license to so...,1:50,10-jul-07,outversion,mark ronson,version [ explicit ],pop,$ 0.99,( c ) 2011 j'adore records,1:50,"july 10 , 2007"
2,534,0,peer pressure ( feat . traci nelson ),snoop dogg,doggumentary,"hip-hop/rap , music , rock , gangsta rap , wes...",$ 1.29,"‰ ãñ 2011 capitol records , llc . all rights r...",4:07,29-mar-11,boom ( ( feat . t-pain ) [ edited ] ),snoop dogg,doggumentary [ edited ],"rap & hip-hop , west coast",$ 1.29,"( c ) 2011 capitol records , llc",3:50,"march 29 , 2011"
3,181,1,stars come out ( tim mason remix ),zedd,stars come out ( remixes ) - ep,"dance , music , electronic , house",$ 1.29,2012 dim mak inc .,5:49,20-may-14,stars come out ( dillon francis remix ),zedd,stars come out [ dillon francis remix ],dance & electronic,$ 1.29,2012 dim mak inc .,4:08,"may 20 , 2014"
4,485,0,jump ( feat . nelly furtado ),flo rida,r.o.o.t.s . ( deluxe version ),"hip-hop/rap , music",$ 1.29,‰ ãñ 2009 atlantic recording corporation for t...,3:28,30-mar-09,"yayo [ feat . brisco , billy blue , ball greez...",flo rida,r.o.o.t.s . ( route of overcoming the struggle...,rap & hip-hop,$ 1.29,"( c ) 2012 motown records , a division of umg ...",7:53,"march 30 , 2009"


In [5]:
test.get_raw_table().head(10)

,id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
0,28,0,lifted ( feat . emeli sand ' © & professor gre...,naughty boy,hotel cabana ( deluxe version ),"pop , music , rock , r & b / soul , contempora...",$ 34.08,2013 naughty boy recordings ltd under exclusiv...,4:15,6-may-14,la la la [ feat . sam smith ],naughty boy,hotel cabana [ explicit ],pop,$ 313.14,( c ) 2014 naughty boy recordings ltd under ex...,3:42,"may 6 , 2014"
1,411,1,better than words,one direction,midnight memories,"pop , music , rock , pop/rock , teen pop , dance",$ 188.56,2013 simco limited under exclusive license to ...,3:27,25-nov-13,better than words,one direction,midnight memories ( deluxe ),pop,$ 924.07,"( c ) 2014 disney enterprises , inc .",3:28,"november 25 , 2013"
2,440,1,who i am with you,chris young,a.m .,"country , music , urban cowboy , contemporary ...",$ 526.36,‰ ãñ 2013 sony music entertainment,3:13,16-sep-13,who i am with you,chris young,a.m .,country,$ 625.93,"( c ) 2010 hollywood records , inc .",3:11,"september 16 , 2013"
3,183,1,the legend of zelda ( electrixx remix ),zedd,the legend of zelda - single,"dance , music",$ 422.87,"2011 bazooka , a division of zooland music gmbh",6:09,27-may-11,the legend of zelda ( electrixx remix ),zedd,the legend of zelda,"electronica , dance & electronic",$ 330.36,"2011 bazooka , a division of zooland music gmbh",6:09,"may 27 , 2011"
4,405,1,see you again,elle king,love stuff,"alternative , music , rock",$ 213.63,"2014 rca records , a division of sony music en...",3:26,13-feb-15,see you again,elle king,love stuff,alternative rock,$ 195.23,( c ) 1996 the island def jam music group,3:26,"february 13 , 2015"
5,438,0,songs like this,carrie underwood,play on,"country , music , pop , adult contemporary , c...",$ 560.99,"‰ ãñ 2009 19 recordings limited , under exclus...",2:37,3-nov-09,this time,carrie underwood,play on,country,$ 54.63,"( p ) 2009 19 recordings limited , under exclu...",3:52,"november 3 , 2009"
6,433,1,skrillex orchestral suite by varien ( bonus tr...,skrillex,bangarang,"dance , music , electronic",$ 22.100,"2012 big beat records , inc. for the united st...",6:54,27-dec-11,skrillex orchestral suite by varien ( bonus tr...,skrillex,bangarang ep [ explicit ],"dance & electronic , dubstep",$ 478.29,elektroshok records,6:54,"december 27 , 2011"
7,146,0,toyfriend ( feat . wynter gordon ) [ continuou...,david guetta,one love ( deluxe version ),"dance , music",$ 361.85,2010 gum prod licence exclusive parlophone mus...,2:51,21-aug-09,dangerous ( feat . sam martin ),david guetta,listen ( deluxe ),dance & electronic,$ 808.38,2015 if moving your mine,3:23,"november 24 , 2014"
8,258,0,o little town of bethlehem,kenny chesney,all i want for christmas is a real good tan,"holiday , music , country , honky tonk , conte...",$ 499.31,2003 bmg music,4:18,4-oct-03,sweet little jesus boy,kenny rogers,christmas greetings,"holiday , christmas , miscellaneous",$ 929.13,"( c ) 2000 capitol records , llc",3:04,"august 17 , 2000"
9,266,0,whateva ( feat . monty ),fetty wap,fetty wap ( deluxe ),"hip-hop/rap , music , rap",$ 553.67,2015 300 entertainment/rgf productions,5:11,,for my team ( feat . monty ) [ clean ],fetty wap,fetty wap ( deluxe ) [ clean ],rap & hip-hop,$ 84.22,"( c ) 2000 mute records ltd. , a bmg company ,...",3:28,"september 25 , 2015"


## Define neural network models

In [28]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')

In [46]:
#run this cell if you want to load pre-trained model
hybrid_model.load_state('models/hybrid_model.pth')
hybrid_model.cuda()

RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/torch/lib/THC/generic/THCTensorCopy.c:20

## Train models

In [56]:
hybrid_model.run_train(
    train,
    validation,
    epochs=10,
    batch_size=16,
    best_save_path='models/hybrid_model.pth',
    pos_neg_ratio=4)

* Number of trainable parameters: 17757810
===>  TRAIN Epoch 1
Finished Epoch 1 || Run Time:    8.1 | Load Time:    1.3 || F1:  98.09 | Prec:  96.25 | Rec: 100.00 || Ex/s:  34.54

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.8 | Load Time:    0.4 || F1:  87.50 | Prec:  87.50 | Rec:  87.50 || Ex/s:  93.54

* Best F1: 87.5
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2
Finished Epoch 2 || Run Time:    8.5 | Load Time:    1.3 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s:  33.04

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.9 | Load Time:    0.4 || F1:  89.80 | Prec:  88.00 | Rec:  91.67 || Ex/s:  85.31

* Best F1: 89.79591836734693
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3
Finished Epoch 3 || Run Time:    8.1 | Load Time:    1.3 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s:  34.45

===>  EVAL Epoch 3
Finished Epoch 3 || Run Time:    0.8 | Load Time:    0.4 || F1:  75.41 | Prec:  62.16 | Rec:  95.83 || Ex/s:  

89.79591836734693

In [109]:
true_predictions = hybrid_model.run_prediction(test)

===>  PREDICT Epoch 2
Finished Epoch 2 || Run Time:    0.5 | Load Time:    0.4 || F1:  86.21 | Prec:  92.59 | Rec:  80.65 || Ex/s: 112.16



In [112]:
true_predictions.head(40)

,match_score
id,
28,0.137302
411,0.912218
440,0.957895
183,0.964156
405,0.995865
438,0.982022
433,0.992676
146,0.148959
258,0.134577


In [111]:
altered_pred = hybrid_model.run_prediction(test)

===>  PREDICT Epoch 2
Finished Epoch 2 || Run Time:    0.3 | Load Time:    0.2 || F1:  83.33 | Prec:  90.91 | Rec:  76.92 || Ex/s:  75.54



In [113]:
altered_pred.head(40)

,match_score
id,
28,0.139293
411,0.897535
440,0.950408
183,0.991464
405,0.995488
438,0.977221
433,0.991583
146,0.150267
258,0.117087


In [117]:
difference = true_predictions.sub(altered_pred)
difference.head(100)

,match_score
id,
10,0.000348
103,NaN
106,0.002505
111,NaN
133,-0.000348
134,NaN
135,NaN
141,-0.004975
143,NaN


In [64]:
test_df = pd.read_csv('sample_data/itunes-amazon/test.csv')
test_df.loc[test_df['label'] == 0, 'left_Time'] = test_df['right_Time']
test_df_negatives = test_df.loc[test_df['label']==0]

In [65]:
test_df_negatives.to_csv('sample_data/itunes-amazon/altered_negatives.csv',index=False)

In [66]:
altered_negatives = dm.data.process_unlabeled('sample_data/itunes-amazon/altered_negatives.csv',
                                           hybrid_model,ignore_columns=['label'])

In [67]:
hybrid_model.run_prediction(altered_negatives,output_attributes=True)

===>  PREDICT Epoch 2
Finished Epoch 2 || Run Time:    0.4 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



,match_score,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
id,,,,,,,,,,,,,,,,,,
28,0.936687,0,Lifted ( feat . Emeli Sand ' © & Professor Gre...,Naughty Boy,Hotel Cabana ( Deluxe Version ),"Pop , Music , Rock , R&B / Soul , Contemporary...",$ 1.29,2013 Naughty Boy Recordings Ltd under exclusiv...,3:42,6-May-14,La La La [ feat . Sam Smith ],Naughty Boy,Hotel Cabana [ Explicit ],Pop,$ 1.29,( C ) 2014 Naughty Boy Recordings Ltd under ex...,3:42,"May 6 , 2014"
438,0.995038,0,Songs Like This,Carrie Underwood,Play On,"Country , Music , Pop , Adult Contemporary , C...",$ 1.29,"‰ ãÑ 2009 19 Recordings Limited , under exclus...",3:52,3-Nov-09,This Time,Carrie Underwood,Play On,Country,$ 1.29,"( P ) 2009 19 Recordings Limited , under exclu...",3:52,"November 3 , 2009"
146,0.814778,0,Toyfriend ( feat . Wynter Gordon ) [ Continuou...,David Guetta,One Love ( Deluxe Version ),"Dance , Music",$ 1.29,2010 Gum Prod licence exclusive Parlophone Mus...,3:23,21-Aug-09,Dangerous ( feat . Sam Martin ),David Guetta,Listen ( Deluxe ),Dance & Electronic,$ 1.29,2015 If Moving Your Mine,3:23,"November 24 , 2014"
258,0.977568,0,O Little Town of Bethlehem,Kenny Chesney,All I Want for Christmas Is a Real Good Tan,"Holiday , Music , Country , Honky Tonk , Conte...",$ 0.99,2003 BMG Music,3:04,4-Oct-03,Sweet Little Jesus Boy,Kenny Rogers,Christmas Greetings,"Holiday , Christmas , Miscellaneous",$ 1.29,"( C ) 2000 Capitol Records , LLC",3:04,"August 17 , 2000"
266,0.254105,0,Whateva ( feat . Monty ),Fetty Wap,Fetty Wap ( Deluxe ),"Hip-Hop/Rap , Music , Rap",Album Only,2015 300 Entertainment/RGF Productions,3:28,NaN,For My Team ( feat . Monty ) [ Clean ],Fetty Wap,Fetty Wap ( Deluxe ) [ Clean ],Rap & Hip-Hop,$ 1.29,"( C ) 2000 Mute Records Ltd. , a BMG Company ,...",3:28,"September 25 , 2015"
237,0.974225,0,If I Am a Stranger ( Live In Amsterdam ),Ryan Adams,Live After Deaf ( Live ) [ Complete Collection ],"Singer/Songwriter , Music , Rock",$ 0.99,2012 PAXAM RECORDS,3:17,2-Jul-12,Firecracker ( Live in Amsterdam ),Ryan Adams,Live After Deaf ( Amsterdam ),"Folk , Rock",$ 0.99,2012 PAXAM RECORDS,3:17,"July 2 , 2012"
205,0.954728,0,Second Chance ( Live Acoustic from Kansas City ),Shinedown,Somewhere In the Stratosphere ( Live ),"Rock , Music , Metal , Alternative , Hard Rock",$ 1.29,2011 Atlantic Recording Corporation for the Un...,8:09,18-Apr-11,Heroes ( Live Acoustic From Kansas City ),Shinedown,Somewhere In The Stratosphere,Rock,$ 1.29,( C ) 2005 MCA Nashville,8:09,"April 18 , 2011"
10,0.725468,0,VHS Outro ( Interlude ),X Ambassadors,VHS,"Alternative , Music , Rock , Adult Alternative",$ 1.29,2015 KIDinaKORNER/Interscope Records,0:24,30-Jun-15,Smoke ( Interlude ) [ Explicit ],X Ambassadors,VHS [ Explicit ],Alternative Rock,$ 1.29,( C ) 2015 KIDinaKORNER/Interscope Records,0:24,"June 30 , 2015"
144,0.411704,0,Toyfriend ( feat . Wynter Gordon ) [ Continuou...,David Guetta,One Love ( Deluxe Version ),"Dance , Music",$ 1.29,2010 Gum Prod licence exclusive Parlophone Mus...,3:45,21-Aug-09,Crossroads ( Album Version ( Edited ) ) [ feat...,David Banner,Certified ( Edited Version ),Rap & Hip-Hop,$ 0.99,"( C ) 2014 Island Records , a division of UMG ...",3:45,"September 20 , 2005"


# Analyze intermediate layers
In this step we want to evaluate the output of intermediate layers. For this purpose we define some utility functions

In [7]:
# class to save intermediate layer output/input
class Hook():
    def __init__(self, module, backward=False):
        if backward==False:
            self.hook = module.register_forward_hook(self.hook_fn)
        else:
            self.hook = module.register_backward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.input = input
        self.output = output
    def close(self):
        self.hook.remove()

In [8]:
def return_layer_input_output(hook_functions,batch,model):
    out= model(batch)
    layer_inputs = []
    layer_outputs = []
    for hook in hook_functions:
        layer_inputs.append(hook.input)
        layer_outputs.append(hook.output)
    return layer_inputs,layer_outputs

## Classifier Evaluation
We want to evaluate the distance between positive and negative example respect to the classifier input

In [9]:
attributes = ['Song_Name','Artist_Name','Album_Name','Genre','Price','CopyRight','Time','Released']

In [31]:
comparators_datasets = dm.data.process(path='sample_data/itunes-amazon/',train='negative_samples.csv',
                            validation='altered_positive_samples.csv',test='positive_samples.csv',
                                       cache='comparator_cache.pth')

In [42]:
from deepmatcher.data import MatchingIterator
batch_size = 8
splits = MatchingIterator.splits(comparators_datasets,batch_size=batch_size,device=-1)

In [43]:
positive_batches = []
for batch in splits[2]:
    positive_batches.append(batch)

In [44]:
altered_positives_batches = []
for batch in splits[1]:
    altered_positives_batches.append(batch)

In [45]:
negative_batches = []
for batch in splits[0]:
    negative_batches.append(batch)

In [15]:
len(positive_batches)

5

In [16]:
summarizers = []
comparators = []
for attr in attributes:
    summarizers.append(hybrid_model.attr_summarizers[attr])
    comparators.append(hybrid_model.attr_comparators[attr])

In [17]:
hookF_summarizer = []
for summ in summarizers:
    hookF_summarizer.append(Hook(summ))
hookF_comparator = []
for comp in comparators:
    hookF_comparator.append(Hook(comp))

In [18]:
classifier = hybrid_model.classifier
hookF_classifier = []
hookF_classifier.append(Hook(classifier))

In [19]:
positives_batch_layer_inputs,positives_batch_layer_outputs = return_layer_input_output(hookF_summarizer,
                                                                                     positive_batches[0],hybrid_model)

In [20]:
altered_batch_layer_inputs,altered_batch_layer_outputs = return_layer_input_output(hookF_summarizer,
                                                                                  altered_positives_batches[0],hybrid_model)

In [26]:
positive_batches[0]

In [22]:
positives_summarizers_left_output = list(map(lambda x: x[0].data,positives_batch_layer_outputs))
positives_summarizers_right_output = list(map(lambda x: x[1].data,positives_batch_layer_outputs))

In [23]:
altered_summarizers_left_output = list(map(lambda x: x[0].data,altered_batch_layer_outputs))
altered_summarizers_right_output = list(map(lambda x:x[1].data,altered_batch_layer_outputs))

In [24]:
negative_classifier_inputs = list(map(lambda x: x[0][0].data,classifier_inputs))

NameError: name 'classifier_inputs' is not defined

In [ ]:
positives_classifier_input = positives_classifier_input[0][0].data

In [ ]:
distance.euclidean( positives_classifier_input[0],negative_classifier_inputs[1][27])

In [ ]:
altered_summarizers_right_output[0]

In [ ]:
def calculate_distance_matrix(summarizers_left_output,summarizers_right_output):
    distance_mat = []
    for i in range(len(summarizers_left_output)):
        distances = []
        for j in range(batch_size):
            l_out = summarizers_left_output[i][j].data
            r_out = summarizers_right_output[i][j].data
            dist = distance.euclidean(l_out,r_out)
            distances.append(dist)
        distance_mat.append(distances)
    distance_mat = np.matrix(distance_mat)
    return distance_mat

In [ ]:
distance_mat_positives = calculate_distance_matrix(positives_summarizers_left_output,positives_summarizers_right_output)
distance_mat_positives = distance_mat_positives.reshape((batch_size,8))

In [ ]:
distances_positives_df = pd.DataFrame(data = distance_mat_positives,columns=attributes)

In [ ]:
distances_positives_df.head(16)

In [ ]:
distances_positives_df.to_csv('distances_positives_batch1.csv',index=False)

In [ ]:
distance_mat_altered = calculate_distance_matrix(altered_summarizers_left_output,altered_summarizers_right_output)
distance_mat_altered = distance_mat_altered.reshape((batch_size,8))

In [ ]:
distances_altered_df = pd.DataFrame(data=distance_mat_altered,columns = attributes)

In [ ]:
distances_altered_df.head()

In [ ]:
distances_altered_df.to_csv('distances_altered_batch1.csv',index=False)

In [ ]:
differences_df = distances_positives_df.subtract(distances_altered_df)

In [ ]:
differences_df.head(16)

## Classifier Evaluation
We want to evaluate the distance between positive and negative example respect to the classifier input

In [189]:
classifier_datasets = dm.data.process(path='sample_data/itunes-amazon/',train='negative_samples.csv',
                            validation='positive_samples.csv',test='all_samples.csv',cache='pcache.pth')

In [190]:
from deepmatcher.data import MatchingIterator
batch_size = 32
splits = MatchingIterator.splits(classifier_datasets,batch_size=batch_size)

In [191]:
negative_batches = []
for batch in splits[0]:
    negative_batches.append(batch)

In [192]:
positive_batches = [] 
for batch in splits[1]:
    positive_batches.append(batch)

In [193]:
all_sample_batches= []
for batch in splits[2]:
    all_sample_batches.append(batch)

In [194]:
classifier = hybrid_model.classifier

In [195]:
hybrid_model.forward(negative_batches[0])

Variable containing:
-0.3846 -1.1417
-0.1488 -1.9789
-0.1380 -2.0486
-0.1393 -2.0403
-0.1590 -1.9170
-0.1685 -1.8637
-0.1994 -1.7105
-0.1610 -1.9058
-0.1555 -1.9379
-0.1531 -1.9523
-0.1337 -2.0781
-0.1374 -2.0527
-0.1345 -2.0727
-0.1625 -1.8971
-0.1384 -2.0462
-0.1588 -1.9187
-0.1534 -1.9504
-0.1668 -1.8735
-0.3113 -1.3187
-0.1639 -1.8891
-0.2356 -1.5612
-0.1672 -1.8712
-0.1840 -1.7832
-0.1454 -2.0002
-0.1678 -1.8676
-0.1705 -1.8529
-0.1742 -1.8332
-0.1799 -1.8037
-0.1590 -1.9170
-0.1300 -2.1046
-0.1820 -1.7934
-0.1764 -1.8220
[torch.cuda.FloatTensor of size 32x2 (GPU 0)]

In [138]:
hookF_classifier = []
hookF_classifier.append(Hook(classifier))

In [163]:
positive_classifier_inputs = []
positive_classifier_outputs = []
for batch in positive_batches:
    classifier_input,classifier_output = return_layer_input_output(hookF_classifier,batch,hybrid_model)
    positive_classifier_inputs.append(classifier_input)
    positive_classifier_outputs.append(classifier_output)

In [141]:
negative_classifier_inputs = []
negative_classifier_outputs = []
for batch in negative_batches:
    classifier_input,classifier_output = return_layer_input_output(hookF_classifier,batch,hybrid_model)
    negative_classifier_inputs.append(classifier_input)
    negative_classifier_outputs.append(classifier_output)

In [164]:
positive_classifier_inputs = list(map(lambda x: x[0][0],positive_classifier_inputs))
positive_classifier_outputs = list(map(lambda x: x[0][0],positive_classifier_outputs))

In [143]:
negative_classifier_inputs = list(map(lambda x: x[0][0],negative_classifier_inputs))
negative_classifier_outputs = list(map(lambda x: x[0][0],negative_classifier_outputs))

In [166]:
positive_classifier_inputs[0][0][0:150]

Variable containing:
-0.0550
 0.1708
 0.2730
-0.1299
-0.0209
 0.1363
 0.2666
 0.0260
 0.2050
-0.1739
 0.1840
-0.1286
 0.3373
-0.1092
 0.2353
 0.2786
 0.1750
 0.2325
-0.1385
 0.1388
 0.3107
-0.0553
 0.1466
 0.0868
-0.2761
 0.1148
 0.2235
 0.0795
 0.0895
 0.3628
 0.1104
 0.1671
-0.3728
 0.2097
 0.0682
 0.1218
 0.1683
-0.0280
 0.3310
 0.2553
 0.2733
 0.1755
 0.3955
 0.1105
-0.0628
 0.0824
-0.1270
 0.1650
 0.0038
 0.1130
-0.0554
 0.1707
 0.2730
-0.1302
-0.0213
 0.1366
 0.2668
 0.0259
 0.2050
-0.1741
 0.1837
-0.1305
 0.3362
-0.1093
 0.2355
 0.2789
 0.1752
 0.2325
-0.1387
 0.1387
 0.3107
-0.0549
 0.1464
 0.0869
-0.2771
 0.1146
 0.2234
 0.0795
 0.0894
 0.3634
 0.1106
 0.1672
-0.3732
 0.2098
 0.0683
 0.1221
 0.1683
-0.0281
 0.3315
 0.2555
 0.2736
 0.1754
 0.3960
 0.1099
-0.0628
 0.0821
-0.1272
 0.1647
 0.0035
 0.1131
 0.0003
 0.0001
 0.0000
 0.0003
 0.0004
 0.0002
 0.0002
 0.0001
 0.0000
 0.0002
 0.0003
 0.0019
 0.0011
 0.0001
 0.0001
 0.0003
 0.0003
 0.0001
 0.0001
 0.0001
 0.0000
 0.0004
 0.

In [167]:
out[0][0]

Variable containing:
 2.3026e-02
 3.5431e-01
-3.9480e-02
-8.7422e-02
 1.1288e-01
-1.1800e-01
 9.5866e-02
 1.9868e-01
-3.8733e-02
-1.2379e-01
 4.5163e-01
 8.8126e-02
 1.7700e-01
 1.7150e-01
 3.8681e-02
-3.1448e-03
 1.4471e-02
 3.2698e-01
-7.1555e-02
 1.7981e-01
-5.7991e-02
-1.0439e-02
 1.7725e-01
-8.2335e-02
 1.4429e-01
 3.0246e-01
 2.4370e-01
-6.0116e-02
 2.9700e-01
 6.4959e-02
-1.1815e-01
-1.2061e-01
-5.1389e-02
-1.3213e-01
 7.9412e-02
-6.3139e-02
 3.3717e-01
 9.1055e-02
 1.1179e-01
-1.9736e-02
 1.0462e-01
 3.0003e-01
 7.4448e-02
-6.9005e-02
 2.6988e-01
 3.1239e-01
 1.2438e-01
-3.2316e-02
-1.3909e-01
-9.4436e-02
 2.2730e-02
 3.5428e-01
-3.9485e-02
-8.7636e-02
 1.1256e-01
-1.1781e-01
 9.6022e-02
 1.9865e-01
-3.8691e-02
-1.2398e-01
 4.5135e-01
 8.6365e-02
 1.7639e-01
 1.7146e-01
 3.8748e-02
-3.0023e-03
 1.4696e-02
 3.2696e-01
-7.1672e-02
 1.7974e-01
-5.7986e-02
-1.0213e-02
 1.7706e-01
-8.2259e-02
 1.4324e-01
 3.0234e-01
 2.4355e-01
-6.0083e-02
 2.9699e-01
 6.5228e-02
-1.1806e-01
-1.2055

## Find attribute more sensible to variation

In [66]:
grads = {}
def save_grad(name):
    def hook(grad):
        grads[name] = grad
    return hook

In [67]:
from torch.nn.functional import softmax
def get_match_score(vec):
    probabilities = softmax(vec,dim=0)
    return probabilities[1]

In [91]:
def gradient(output,vector_index,gradient_map,attribute):
    output[vector_index,1].backward(retain_graph=True)
    gradient = gradient_map['classifier'][vector_index]
    start_index = attribute *150
    end_index = start_index+150
    partial_derivative = Variable(torch.cuda.FloatTensor(1200).fill_(0))
    partial_derivative[start_index:end_index] = gradient[start_index:end_index]
    return partial_derivative

In [92]:
def find_smallest_variation_to_change(layer,input_matrix, vector_index,grad_maps,attribute):
    input_matrix_copy = input_matrix.clone()
    input_matrix_copy.register_hook(save_grad('classifier'))
    x0= input_matrix_copy[vector_index]
    match_probability = get_match_score(layer.forward(input_matrix_copy)[vector_index])
    xi = x0
    sum_ri = Variable(torch.cuda.FloatTensor(1200).fill_(0))
    iteration = 0
    while(get_match_score(layer.forward(input_matrix_copy)[vector_index]).data[0]<=0.5
          or iteration>100):
        output = layer.forward(input_matrix_copy)
        current_match_score = get_match_score(output[vector_index])
        current_gradient = gradient(output,vector_index,grad_maps,attribute)
        ri = (current_match_score/torch.norm(current_gradient)**2) * current_gradient
        xi = xi+ri
        input_matrix_copy[vector_index].data = input_matrix_copy[vector_index].data.copy_(xi.data)
        sum_ri += ri
        iteration+=1
    if(iteration>100):
        print('Can\'t converge')
    return iteration,sum_ri

In [79]:
classifier_datasets = dm.data.process(path='sample_data/itunes-amazon/',train='negative_samples.csv',
                            validation='negative_samples.csv',test='all_samples.csv',cache='pcache.pth')

In [85]:
from deepmatcher.data import MatchingIterator
batch_size = 32
splits = MatchingIterator.splits(classifier_datasets,batch_size=batch_size)

In [86]:
negative_batches = []
for batch in splits[0]:
    negative_batches.append(batch)

In [87]:
negative_classifier_inputs = []
negative_classifier_outputs = []
for batch in negative_batches:
    classifier_input,classifier_output = return_layer_input_output(hookF_classifier,batch,hybrid_model)
    negative_classifier_inputs.append(classifier_input)
    negative_classifier_outputs.append(classifier_output)

In [88]:
negative_classifier_inputs = list(map(lambda x: x[0][0],negative_classifier_inputs))
negative_classifier_outputs = list(map(lambda x: x[0][0],negative_classifier_outputs))

In [93]:
attribute_lenght= len(attributes)
variation_list = []
current_sample = 0
for batch in negative_classifier_inputs:
    for index in range(len(batch)):
        variation_norms = []
        for j,attribute in enumerate(attributes):
            print('Processing sample {} with attribute {}'.format(current_sample,attribute))
            it,variation = find_smallest_variation_to_change(hybrid_model.classifier,batch,index,grads,j)
            variation_norms.append(torch.norm(variation))
        variation_list.append(variation_norms)
        current_sample+=1

Processing sample 0 with attribute Song_Name
Processing sample 0 with attribute Artist_Name
Processing sample 0 with attribute Album_Name
Processing sample 0 with attribute Genre
Processing sample 0 with attribute Price
Processing sample 0 with attribute CopyRight
Processing sample 0 with attribute Time
Processing sample 0 with attribute Released
Processing sample 1 with attribute Song_Name
Processing sample 1 with attribute Artist_Name
Processing sample 1 with attribute Album_Name
Processing sample 1 with attribute Genre
Processing sample 1 with attribute Price
Processing sample 1 with attribute CopyRight
Processing sample 1 with attribute Time
Processing sample 1 with attribute Released
Processing sample 2 with attribute Song_Name
Processing sample 2 with attribute Artist_Name
Processing sample 2 with attribute Album_Name
Processing sample 2 with attribute Genre
Processing sample 2 with attribute Price
Processing sample 2 with attribute CopyRight
Processing sample 2 with attribute Ti

Processing sample 23 with attribute Album_Name
Processing sample 23 with attribute Genre
Processing sample 23 with attribute Price
Processing sample 23 with attribute CopyRight
Processing sample 23 with attribute Time
Processing sample 23 with attribute Released
Processing sample 24 with attribute Song_Name
Processing sample 24 with attribute Artist_Name
Processing sample 24 with attribute Album_Name
Processing sample 24 with attribute Genre
Processing sample 24 with attribute Price
Processing sample 24 with attribute CopyRight
Processing sample 24 with attribute Time
Processing sample 24 with attribute Released
Processing sample 25 with attribute Song_Name
Processing sample 25 with attribute Artist_Name
Processing sample 25 with attribute Album_Name
Processing sample 25 with attribute Genre
Processing sample 25 with attribute Price
Processing sample 25 with attribute CopyRight
Processing sample 25 with attribute Time
Processing sample 25 with attribute Released
Processing sample 26 wi

Processing sample 46 with attribute Album_Name
Processing sample 46 with attribute Genre
Processing sample 46 with attribute Price
Processing sample 46 with attribute CopyRight
Processing sample 46 with attribute Time
Processing sample 46 with attribute Released
Processing sample 47 with attribute Song_Name
Processing sample 47 with attribute Artist_Name
Processing sample 47 with attribute Album_Name
Processing sample 47 with attribute Genre
Processing sample 47 with attribute Price
Processing sample 47 with attribute CopyRight
Processing sample 47 with attribute Time
Processing sample 47 with attribute Released
Processing sample 48 with attribute Song_Name
Processing sample 48 with attribute Artist_Name
Processing sample 48 with attribute Album_Name
Processing sample 48 with attribute Genre
Processing sample 48 with attribute Price
Processing sample 48 with attribute CopyRight
Processing sample 48 with attribute Time
Processing sample 48 with attribute Released
Processing sample 49 wi

Processing sample 69 with attribute Album_Name
Processing sample 69 with attribute Genre
Processing sample 69 with attribute Price
Processing sample 69 with attribute CopyRight
Processing sample 69 with attribute Time
Processing sample 69 with attribute Released
Processing sample 70 with attribute Song_Name
Processing sample 70 with attribute Artist_Name
Processing sample 70 with attribute Album_Name
Processing sample 70 with attribute Genre
Processing sample 70 with attribute Price
Processing sample 70 with attribute CopyRight
Processing sample 70 with attribute Time
Processing sample 70 with attribute Released
Processing sample 71 with attribute Song_Name
Processing sample 71 with attribute Artist_Name
Processing sample 71 with attribute Album_Name
Processing sample 71 with attribute Genre
Processing sample 71 with attribute Price
Processing sample 71 with attribute CopyRight
Processing sample 71 with attribute Time
Processing sample 71 with attribute Released
Processing sample 72 wi

Processing sample 92 with attribute Album_Name
Processing sample 92 with attribute Genre
Processing sample 92 with attribute Price
Processing sample 92 with attribute CopyRight
Processing sample 92 with attribute Time
Processing sample 92 with attribute Released
Processing sample 93 with attribute Song_Name
Processing sample 93 with attribute Artist_Name
Processing sample 93 with attribute Album_Name
Processing sample 93 with attribute Genre
Processing sample 93 with attribute Price
Processing sample 93 with attribute CopyRight
Processing sample 93 with attribute Time
Processing sample 93 with attribute Released
Processing sample 94 with attribute Song_Name
Processing sample 94 with attribute Artist_Name
Processing sample 94 with attribute Album_Name
Processing sample 94 with attribute Genre
Processing sample 94 with attribute Price
Processing sample 94 with attribute CopyRight
Processing sample 94 with attribute Time
Processing sample 94 with attribute Released
Processing sample 95 wi

...

Processing sample 295 with attribute Time
Processing sample 295 with attribute Released
Processing sample 296 with attribute Song_Name
Processing sample 296 with attribute Artist_Name
Processing sample 296 with attribute Album_Name
Processing sample 296 with attribute Genre
Processing sample 296 with attribute Price
Processing sample 296 with attribute CopyRight
Processing sample 296 with attribute Time
Processing sample 296 with attribute Released
Processing sample 297 with attribute Song_Name
Processing sample 297 with attribute Artist_Name
Processing sample 297 with attribute Album_Name
Processing sample 297 with attribute Genre
Processing sample 297 with attribute Price
Processing sample 297 with attribute CopyRight
Processing sample 297 with attribute Time
Processing sample 297 with attribute Released
Processing sample 298 with attribute Song_Name
Processing sample 298 with attribute Artist_Name
Processing sample 298 with attribute Album_Name
Processing sample 298 with attribute G

Processing sample 318 with attribute Album_Name
Processing sample 318 with attribute Genre
Processing sample 318 with attribute Price
Processing sample 318 with attribute CopyRight
Processing sample 318 with attribute Time
Processing sample 318 with attribute Released
Processing sample 319 with attribute Song_Name
Processing sample 319 with attribute Artist_Name
Processing sample 319 with attribute Album_Name
Processing sample 319 with attribute Genre
Processing sample 319 with attribute Price
Processing sample 319 with attribute CopyRight
Processing sample 319 with attribute Time
Processing sample 319 with attribute Released
Processing sample 320 with attribute Song_Name
Processing sample 320 with attribute Artist_Name
Processing sample 320 with attribute Album_Name
Processing sample 320 with attribute Genre
Processing sample 320 with attribute Price
Processing sample 320 with attribute CopyRight
Processing sample 320 with attribute Time
Processing sample 320 with attribute Released
P

Processing sample 340 with attribute Time
Processing sample 340 with attribute Released
Processing sample 341 with attribute Song_Name
Processing sample 341 with attribute Artist_Name
Processing sample 341 with attribute Album_Name
Processing sample 341 with attribute Genre
Processing sample 341 with attribute Price
Processing sample 341 with attribute CopyRight
Processing sample 341 with attribute Time
Processing sample 341 with attribute Released
Processing sample 342 with attribute Song_Name
Processing sample 342 with attribute Artist_Name
Processing sample 342 with attribute Album_Name
Processing sample 342 with attribute Genre
Processing sample 342 with attribute Price
Processing sample 342 with attribute CopyRight
Processing sample 342 with attribute Time
Processing sample 342 with attribute Released
Processing sample 343 with attribute Song_Name
Processing sample 343 with attribute Artist_Name
Processing sample 343 with attribute Album_Name
Processing sample 343 with attribute G

Processing sample 363 with attribute Album_Name
Processing sample 363 with attribute Genre
Processing sample 363 with attribute Price
Processing sample 363 with attribute CopyRight
Processing sample 363 with attribute Time
Processing sample 363 with attribute Released
Processing sample 364 with attribute Song_Name
Processing sample 364 with attribute Artist_Name
Processing sample 364 with attribute Album_Name
Processing sample 364 with attribute Genre
Processing sample 364 with attribute Price
Processing sample 364 with attribute CopyRight
Processing sample 364 with attribute Time
Processing sample 364 with attribute Released
Processing sample 365 with attribute Song_Name
Processing sample 365 with attribute Artist_Name
Processing sample 365 with attribute Album_Name
Processing sample 365 with attribute Genre
Processing sample 365 with attribute Price
Processing sample 365 with attribute CopyRight
Processing sample 365 with attribute Time
Processing sample 365 with attribute Released
P

Processing sample 386 with attribute Artist_Name
Processing sample 386 with attribute Album_Name
Processing sample 386 with attribute Genre
Processing sample 386 with attribute Price
Processing sample 386 with attribute CopyRight
Processing sample 386 with attribute Time
Processing sample 386 with attribute Released
Processing sample 387 with attribute Song_Name
Processing sample 387 with attribute Artist_Name
Processing sample 387 with attribute Album_Name
Processing sample 387 with attribute Genre
Processing sample 387 with attribute Price
Processing sample 387 with attribute CopyRight
Processing sample 387 with attribute Time
Processing sample 387 with attribute Released
Processing sample 388 with attribute Song_Name
Processing sample 388 with attribute Artist_Name
Processing sample 388 with attribute Album_Name
Processing sample 388 with attribute Genre
Processing sample 388 with attribute Price
Processing sample 388 with attribute CopyRight
Processing sample 388 with attribute Tim

In [75]:
attribute_lenght= len(attributes)
variation_list = []
current_sample = 0
for batch in positive_classifier_inputs:
    for index in range(len(batch)):
        variation_norms = []
        for j,attribute in enumerate(attributes):
            print('Processing sample {} with attribute {}'.format(current_sample,attribute))
            it,variation = find_smallest_variation_to_change(hybrid_model.classifier,batch,index,grads,j)
            variation_norms.append(torch.norm(variation))
        variation_list.append(variation_norms)
        current_sample+=1

Processing sample 0 with attribute Song_Name
Processing sample 0 with attribute Artist_Name
Processing sample 0 with attribute Album_Name
Processing sample 0 with attribute Genre
Processing sample 0 with attribute Price
Processing sample 0 with attribute CopyRight
Processing sample 0 with attribute Time
Processing sample 0 with attribute Released
Processing sample 1 with attribute Song_Name
Processing sample 1 with attribute Artist_Name
Processing sample 1 with attribute Album_Name
Processing sample 1 with attribute Genre
Processing sample 1 with attribute Price
Processing sample 1 with attribute CopyRight
Processing sample 1 with attribute Time
Processing sample 1 with attribute Released
Processing sample 2 with attribute Song_Name
Processing sample 2 with attribute Artist_Name
Processing sample 2 with attribute Album_Name
Processing sample 2 with attribute Genre
Processing sample 2 with attribute Price
Processing sample 2 with attribute CopyRight
Processing sample 2 with attribute Ti

Processing sample 24 with attribute Album_Name
Processing sample 24 with attribute Genre
Processing sample 24 with attribute Price
Processing sample 24 with attribute CopyRight
Processing sample 24 with attribute Time
Processing sample 24 with attribute Released
Processing sample 25 with attribute Song_Name
Processing sample 25 with attribute Artist_Name
Processing sample 25 with attribute Album_Name
Processing sample 25 with attribute Genre
Processing sample 25 with attribute Price
Processing sample 25 with attribute CopyRight
Processing sample 25 with attribute Time
Processing sample 25 with attribute Released
Processing sample 26 with attribute Song_Name
Processing sample 26 with attribute Artist_Name
Processing sample 26 with attribute Album_Name
Processing sample 26 with attribute Genre
Processing sample 26 with attribute Price
Processing sample 26 with attribute CopyRight
Processing sample 26 with attribute Time
Processing sample 26 with attribute Released
Processing sample 27 wi

Processing sample 47 with attribute Album_Name
Processing sample 47 with attribute Genre
Processing sample 47 with attribute Price
Processing sample 47 with attribute CopyRight
Processing sample 47 with attribute Time
Processing sample 47 with attribute Released
Processing sample 48 with attribute Song_Name
Processing sample 48 with attribute Artist_Name
Processing sample 48 with attribute Album_Name
Processing sample 48 with attribute Genre
Processing sample 48 with attribute Price
Processing sample 48 with attribute CopyRight
Processing sample 48 with attribute Time
Processing sample 48 with attribute Released
Processing sample 49 with attribute Song_Name
Processing sample 49 with attribute Artist_Name
Processing sample 49 with attribute Album_Name
Processing sample 49 with attribute Genre
Processing sample 49 with attribute Price
Processing sample 49 with attribute CopyRight
Processing sample 49 with attribute Time
Processing sample 49 with attribute Released
Processing sample 50 wi

Processing sample 70 with attribute Album_Name
Processing sample 70 with attribute Genre
Processing sample 70 with attribute Price
Processing sample 70 with attribute CopyRight
Processing sample 70 with attribute Time
Processing sample 70 with attribute Released
Processing sample 71 with attribute Song_Name
Processing sample 71 with attribute Artist_Name
Processing sample 71 with attribute Album_Name
Processing sample 71 with attribute Genre
Processing sample 71 with attribute Price
Processing sample 71 with attribute CopyRight
Processing sample 71 with attribute Time
Processing sample 71 with attribute Released
Processing sample 72 with attribute Song_Name
Processing sample 72 with attribute Artist_Name
Processing sample 72 with attribute Album_Name
Processing sample 72 with attribute Genre
Processing sample 72 with attribute Price
Processing sample 72 with attribute CopyRight
Processing sample 72 with attribute Time
Processing sample 72 with attribute Released
Processing sample 73 wi

Processing sample 93 with attribute Album_Name
Processing sample 93 with attribute Genre
Processing sample 93 with attribute Price
Processing sample 93 with attribute CopyRight
Processing sample 93 with attribute Time
Processing sample 93 with attribute Released
Processing sample 94 with attribute Song_Name
Processing sample 94 with attribute Artist_Name
Processing sample 94 with attribute Album_Name
Processing sample 94 with attribute Genre
Processing sample 94 with attribute Price
Processing sample 94 with attribute CopyRight
Processing sample 94 with attribute Time
Processing sample 94 with attribute Released
Processing sample 95 with attribute Song_Name
Processing sample 95 with attribute Artist_Name
Processing sample 95 with attribute Album_Name
Processing sample 95 with attribute Genre
Processing sample 95 with attribute Price
Processing sample 95 with attribute CopyRight
Processing sample 95 with attribute Time
Processing sample 95 with attribute Released
Processing sample 96 wi

Processing sample 115 with attribute Released
Processing sample 116 with attribute Song_Name
Processing sample 116 with attribute Artist_Name
Processing sample 116 with attribute Album_Name
Processing sample 116 with attribute Genre
Processing sample 116 with attribute Price
Processing sample 116 with attribute CopyRight
Processing sample 116 with attribute Time
Processing sample 116 with attribute Released
Processing sample 117 with attribute Song_Name
Processing sample 117 with attribute Artist_Name
Processing sample 117 with attribute Album_Name
Processing sample 117 with attribute Genre
Processing sample 117 with attribute Price
Processing sample 117 with attribute CopyRight
Processing sample 117 with attribute Time
Processing sample 117 with attribute Released
Processing sample 118 with attribute Song_Name
Processing sample 118 with attribute Artist_Name
Processing sample 118 with attribute Album_Name
Processing sample 118 with attribute Genre
Processing sample 118 with attribute 

In [94]:
variations_list = []
for variationl in variation_list:
    variations_list.append(list(map(lambda x:x.data[0],variationl)))

In [95]:
variationdf = pd.DataFrame(data=variations_list,columns=attributes)
variationdf.head()

,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,11.272275,8.526607,12.307614,14.973045,9.552113,13.978187,7.660950,13.972967
1,4.758212,6.041291,5.462882,7.817120,5.289720,7.359730,4.087255,8.238219
2,1.379403,1.327967,1.981606,2.916867,1.480839,2.958233,1.172890,3.147770
3,8.304395,10.255034,9.371615,13.002915,9.428836,12.465248,7.955159,13.298765
4,10.593219,8.224025,10.800851,13.141464,8.941920,12.446552,7.386593,12.500021


In [96]:
variationdf.to_csv('negative_samples_ri_vectors_no_bias_model.csv',index=False,sep=';')

In [ ]:
def euclidean_distance_with_max_difference_dimension(v,q):
    distance = 0
    max_difference = -1
    max_difference_dimension = 0
    for idx,vi in enumerate(v):
        qi = q[idx]
        diff = qi-vi
        if(diff>max_difference):
            max_difference = diff
            max_difference_dimension = idx
        distance += (diff)**2
    return math.sqrt(distance),max_difference_dimension

In [ ]:
euclidean_distance_with_max_difference_dimension(positive_classifier_inputs[1][10],negative_classifier_inputs[1][10])

In [ ]:
distance.euclidean(positive_classifier_inputs[1][10],negative_classifier_inputs[1][10])

In [ ]:
#in input abbiamo due liste di vettori di un batch
def calculate_closer_vector(pos_vector_list,neg_vector_list):
    #mi salvo l'indice del vettore più vicino come chiave
    closer_vectors = []
    for curr_pos_batch in pos_vector_list:
        for curr_positive in curr_pos_batch:
            print('proccessing vector')
            current_min = 100000000
            index = 1
            closer_index = -1
            max_dim = 0
            for batch in neg_vector_list:
                for curr_negative in batch:
                    curr_distance,dim_max = euclidean_distance_with_max_difference_dimension(curr_positive,curr_negative)
                    if(curr_distance<current_min):
                        current_min=curr_distance
                        closer_index = index
                        max_dim = dim_max
                    index +=1
            closer_vectors.append((closer_index, current_min,dim_max))
    return closer_vectors

In [ ]:
closer_vectors = calculate_closer_vector(positive_classifier_inputs,negative_classifier_inputs)

In [ ]:
closer_vectors_df = pd.DataFrame(data=closer_vectors,columns=['Closer_vector','distance','max_difference_dimension'])

In [ ]:
closer_vectors_df.head(10)

In [ ]:
closer_vectors_df.to_csv('classifier_distances.csv',index=False)

In [ ]:
closer_vectors_df['Closer_vector'].nunique()

# Attribute variation analysis

In [93]:
dir(test)

['CacheStaleException',
 '__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_set_attributes',
 'all_left_fields',
 'all_right_fields',
 'all_text_fields',
 'canonical_text_fields',
 'column_naming',
 'compute_metadata',
 'corresponding_field',
 'download',
 'examples',
 'fields',
 'finalize_metadata',
 'get_raw_table',
 'id_field',
 'label_field',
 'load_cache',
 'path',
 'restore_data',
 'save_cache',
 'sort_key',
 'split',
 'splits',
 'state_args_compatibility',
 'text_fields',
 'vocabs']